In [5]:
import functools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from typing import Dict, Any, Optional, List, Tuple
from transformers import pipeline

In [1]:
dataset = pd.read_csv("data/train.csv")

NameError: name 'pd' is not defined

## NLTK-based word frequency inspection

In [9]:
classifier = pipeline(
    "zero-shot-classification", model="facebook/bart-large-mnli", device_map="auto"
)

Device set to use cuda:0


In [12]:
class AiData(torch.utils.data.Dataset):
    def __init__(self, df):
        self.data = df["ai_text"].tolist()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [16]:
idx = 0
n_entries = 100
label_rows = []
h_batch_size = 128
diff = 0

sample_sm = df.sample(n_entries)
data = AiData(sample_sm)

for r in classifier(data, combined_tags, batch_size=h_batch_size):

    print(f"\rPROGRESS: {idx + 1} / {n_entries}", end="")
    row = [r["sequence"]]

    for c in combined_tags:
        row.append(r["scores"][r["labels"].index(c)])

    label_rows.append(row)
    idx += 1

    # Purge cache once in a while
    if idx - diff > h_batch_size * 8:
        torch.cuda.empty_cache()
        diff = idx


df_labels = pd.DataFrame(label_rows, columns=["ai_text"] + combined_tags)
df_labels.to_csv(f"labels.csv", index=False)

PROGRESS: 100 / 100

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

NameError: name 'df_labels' is not defined